Here is a sample script to do geometry optimization in Gaussian on a remote machine

In [1]:
from MolMod.BasicTools import *
from MolMod.SSHTools import *
from MolMod.MolMod import *
mm=MolMod()
mm.ssh=SSHTools()
import warnings
warnings.filterwarnings(action='ignore',module='.*paramiko.*')

In [2]:
# Setup SSH parameters
mm.ssh.setup("cmsVM")
#mm.ssh.setup(user="a.zhugayevych",host="10.30.16.165",ppk="../ppk/cmsVM.ppk",pbs="SLURM")

In [3]:
# Set molecule name and method tag
mol="C4H5N"     # pyrrole
tag="_B3LYPp2"  # B3LYP/6-31g (p2 = double-zeta Pople basis set)
fn=mol+tag+"_em"

In [4]:
# Read initial geometry
Atoms=mm.ReadXYZ(mol+"_exp.xyz")

In [5]:
mm.WriteInput(fn+".gau",Atoms,keyline="B3LYP 6-31g opt",output=["T"],printout=True)

#T B3LYP 6-31g opt


0 1
N  1.121800 0.000000 0.000000
C  0.334100 1.120900 0.000000
C  0.334100 -1.120900 0.000000
C  -0.984700 0.707600 0.000000
C  -0.984700 -0.707600 0.000000
H  2.117800 0.000000 0.000000
H  0.761400 2.108400 0.000000
H  0.761400 -2.108400 0.000000
H  -1.842900 1.356600 0.000000
H  -1.842900 -1.356600 0.000000





0

In [6]:
# SubmitJob
#mm.SubmitJob(fn+".gau",ppn=4,mem=4*3,q='fastio',printout=True,overwrite=False,test=False)
mm.SubmitJob(fn+".gau",ppn=4,printout=True)

sbatch -J C4H5N_B3LYPp2_em -c 4 -t 1:00:00 gaussian/_pbs


0

In [7]:
# Check job
mm.ssh.qstat()

command: squeue -u a.zhugayevych
output:
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                47     class C4H5N_B3 a.zhugay  R       0:05      1 ccmm-cest-2019


In [8]:
# Delete job if needed
#mm.ssh.run("qdel 990156")

In [9]:
# Download job
mm.DownloadJob(fn)

0

In [10]:
# Extract optimized geometry
Atoms1=mm.ReadAtoms(fn+".out")
print(Atoms1)

[['N', array([-0.      ,  0.      ,  1.129845])], ['C', array([0.      , 1.134027, 0.331508])], ['C', array([-0.      , -1.134027,  0.331508])], ['C', array([ 0.      ,  0.715786, -0.986193])], ['C', array([-0.      , -0.715786, -0.986193])], ['H', array([-0.      ,  0.      ,  2.135892])], ['H', array([0.      , 2.123532, 0.759183])], ['H', array([-0.      , -2.123532,  0.759183])], ['H', array([ 0.      ,  1.358587, -1.853473])], ['H', array([-0.      , -1.358587, -1.853473])]]


In [11]:
# Save it in xyz-file
mm.WriteXYZ(fn+".xyz",Atoms1)